Scenario
load our libs
load our data
eda
logistic regression
evaluation model
diplay result
model for prediction


LLR test p value
coefficients
model hypothesis
h0 the model is no better than null model
h1 the model id significantly better than null mode

h0: Bi is not signifacnt (No relationship)
H1 : Bi is signifacnt (there is relationship)

T test


Sample size of 30 cases

# Task
Develop a Jupyter Notebook to analyze the "cardholderstudy.csv" dataset. The notebook should:
1.  Load the dataset and perform initial data inspection (head, info, shape, missing values).
2.  Conduct a comprehensive Exploratory Data Analysis (EDA) with appropriate univariate and multivariate visualizations.
3.  Build a logistic regression model to predict the 'upgrade' variable based on other features.
4.  Display, analyze, and interpret the results of the logistic regression model, including model verification and the significance of individual variables.
5.  Demonstrate how to use the model to make predictions, showing both the percentage likelihood and the binary output.
6.  Include markdown cells to explain each step of the process.

## Load and inspect data

### Subtask:
Load the `cardholderstudy.csv` dataset into a pandas DataFrame. Inspect the first few rows, check for missing values, and examine the data types and shape of the DataFrame.


**Reasoning**:
This code block will import the pandas library, load the specified CSV file into a DataFrame, and then display the initial requested information: the head, info, and shape of the DataFrame, which aligns with the subtask's instructions.



In [ ]:
import numpy as np

import pandas as pd

df = pd.read_csv('/content/cardholderstudy.csv')

print("First 5 rows of the DataFrame:")
display(df.head())

print("\nDataFrame Info:")
df.info()

print("\nDataFrame Shape:")
print(df.shape)

First 5 rows of the DataFrame:


,Upgraded,Purchases,Extra Cards
0,0,32.1007,0
1,1,34.3706,1
2,0,4.8749,0
3,0,8.1263,0
4,0,12.9783,0



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Upgraded     30 non-null     int64  
 1   Purchases    30 non-null     float64
 2   Extra Cards  30 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 852.0 bytes

DataFrame Shape:
(30, 3)


## Exploratory data analysis (EDA)
